# D&D Dice Fraud Detection Software: Neural Network Training

## 0. Import

NOTE: Make sure to switch Python / Anaconda environments to the TensorFlow one.

In [1]:
import numpy as np
import tensorflow as tf
print('This is the Dice Fraud Detection Neural Network training script.')

This is the Dice Fraud Detection Neural Network training script.


## 1. Load Dataset and Train/Validate/Test Split

In [2]:
with np.load("allDataShuffled.npz") as file:
    inputs = file['inputs'].astype(np.float)
    targets=file['targets'].astype(np.float)
Ntot=np.shape(targets)[0]
print('File loaded successfully, now training on a total of {0} datapoints'.format(Ntot))

#loads the file from the .npz, the savez method allows us to save multiple arrays with a key we define
#which is handy when saving objects coming from pandas DataFrames

File loaded successfully, now training on a total of 100000 datapoints


In [3]:
TRAIN_DATA_FRACTION=0.8
VALIDATE_DATA_FRACTION=0.1
TEST_DATA_FRACTION=0.1
#defines the fractions used for train/test/splitting. Note that these are technically model metaparameters.

In [4]:
train_inputs = inputs[:int(np.ceil(TRAIN_DATA_FRACTION *Ntot))]
validate_inputs = inputs[int(np.ceil(TRAIN_DATA_FRACTION *Ntot)):int(np.ceil((TRAIN_DATA_FRACTION+VALIDATE_DATA_FRACTION)*Ntot))]
test_inputs=inputs[int(np.ceil((TRAIN_DATA_FRACTION+VALIDATE_DATA_FRACTION)*Ntot)):]

train_targets = targets[:int(np.ceil(TRAIN_DATA_FRACTION *Ntot))]
validate_targets = targets[int(np.ceil(TRAIN_DATA_FRACTION *Ntot)):int(np.ceil((TRAIN_DATA_FRACTION+VALIDATE_DATA_FRACTION)*Ntot))]
test_targets=targets[int(np.ceil((TRAIN_DATA_FRACTION+VALIDATE_DATA_FRACTION)*Ntot)):]
print('Batching process complete')
#perform train/validate/test splitting, we assume the data is already shuffled.

Batching process complete


## 2. Define Neural Network and its Metaparameters

Do not, I repeat, do **NOT** mess with the Metaparameters and run the training on the same data set. Create an entirely new batch of data points with the data generation script, to avoir all possibility of overfitting.

### a. Algorithmics

In [5]:
INPUT_SIZE=6
#This is fixed by our requirements: one dice set is 6 numbers
TRAIN_RATE=0.0001
#train rate can be small if using an early stopping mechanism
OPTIMIZER=tf.keras.optimizers.Adam(learning_rate=TRAIN_RATE)
#Adam has a momentum term that accelerates the computation but is bad for shallow minima.
METRICS=['accuracy']
#get feedback for the model's accuracy as training goes on
BATCH_SIZE=1000
#to match with the size of the dataset. 
MAX_EPOCHS=200
#to match with the train rate
CALLBACKS= tf.keras.callbacks.EarlyStopping(patience=3)
#sets up an early stopping mechanism that interrupts training if loss stops decreasing.
#patience parameter allows for some fluctuation up and down of the parameter before a full stop, useful for batch training

### b. Network design

In [ ]:
HIDDEN_LAYER_SIZE=200
#height of each hidden layer.
HIDDEN_LAYER_NUMBER=5
#Number of hidden layers before the final output layer
LAYER_SEQUENCE=[tf.keras.layers.Dense(HIDDEN_LAYER_SIZE,activation='relu') for i in range(HIDDEN_LAYER_NUMBER)]
#Defines the hidden layer sequence based on previous parameters. 
#Activation functions are all Rectified Linear Unit, which causes fewer problems than a basic sigmoid
#But requires normalisation, hence:
OUTPUT_SIZE=2
LAYER_SEQUENCE.append(tf.keras.layers.Dense(OUTPUT_SIZE, activation='softmax'))
#Final output layer using a softmax i.e. sigmoidal output to create a unit vector
LOSS='sparse_categorical_crossentropy'
#the correct loss function for a one-hot encoded output layer, i.e. a categorisation problem

#Note that technically we only have one output category by elimination, the following is more suited for a binary problem
#OUTPUT_SIZE=1
#LAYER_SEQUENCE.append(tf.keras.layers.Dense(OUTPUT_SIZE, activation='sigmoid'))
#LOSS='binary_crossentropy'


### c. Create and compile

In [6]:
nnmodel=tf.keras.Sequential(LAYER_SEQUENCE)

nnmodel.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
print('Model compiled, ready to train')

Model compiled, ready to train


## 3. Train Network

In [7]:
nnmodel.fit(train_inputs,train_targets,batch_size=BATCH_SIZE,epochs=MAX_EPOCHS,
          callbacks=[CALLBACKS],
          validation_data=(validate_inputs,validate_targets))



Train on 80000 samples, validate on 10000 samples
Epoch 1/200
80000/80000 [==============================] - 3s 39us/sample - loss: 0.6717 - accuracy: 0.5586 - val_loss: 0.6610 - val_accuracy: 0.5608
Epoch 2/200
80000/80000 [==============================] - 2s 27us/sample - loss: 0.6436 - accuracy: 0.6070 - val_loss: 0.6346 - val_accuracy: 0.6150
Epoch 3/200
80000/80000 [==============================] - 2s 26us/sample - loss: 0.6352 - accuracy: 0.6115 - val_loss: 0.6322 - val_accuracy: 0.6089
Epoch 4/200
80000/80000 [==============================] - 2s 25us/sample - loss: 0.6297 - accuracy: 0.6179 - val_loss: 0.6278 - val_accuracy: 0.6164
Epoch 5/200
80000/80000 [==============================] - 2s 25us/sample - loss: 0.6274 - accuracy: 0.6210 - val_loss: 0.6276 - val_accuracy: 0.6181
Epoch 6/200
80000/80000 [==============================] - 2s 24us/sample - loss: 0.6254 - accuracy: 0.6222 - val_loss: 0.6291 - val_accuracy: 0.6161
Epoch 7/200
80000/80000 [=========================

## 4. Test Model

Again, do **NOT** run this until you have finished fiddling with hyperparameters, or re-generate a dataset.

In [12]:
test_loss,test_accuracy = nnmodel.evaluate(test_inputs,test_targets)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [13]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.44. Test accuracy: 78.58%


In [36]:
testpredict=np.array([18,16,16,14,13,13])
np.random.shuffle(testpredict)
testpredict=testpredict.reshape(1,6)
print(testpredict)

[[13 14 16 13 18 16]]


In [37]:
output=nnmodel.predict(testpredict)
output

array([[0.47950307, 0.5204969 ]], dtype=float32)

## 5. Export Model